## Часть 1

PySpark

1.       Попробуем использовать возможности Spark для анализа данных clickstream пользователей новостного Интернет-портала.

* Создадим схему будущего фрейма данных. Схема должна включать следующие атрибуты:

        ·   id -  уникальный идентификатор посетителя сайта. Тип – последовательность чисел фиксированной длины. Данное поле не является первичным ключом.

        ·   timestamp – дата и время события в формате unix timestamp.

        ·   type – тип события, значение из списка (факт посещения(visit), клик по визуальному элементу страницы(click), скролл(scroll), перед на другую страницу(move)).

        ·   page_id – id текущей страницы. Тип - последовательность чисел фиксированной длины.

        ·   tag – каждая страница с новостью размечается редакцией специальными тегами, которые отражают тематику конкретной новости со страницы. Возможный список тематик: политика, спорт, медицина и т.д.

        ·   sign – наличие у пользователя личного кабинета. Значения – True/False.
        

* Создадим датафрейм с описанной выше схемой данных.

* Наполним датафрейм данными. 

        Пример: (12345, 1667627426, "click", 101, "Sport”, False)

* Подзадачи:

        ·   Вывести топ-5 самых активных посетителей сайта

        ·   Посчитать процент посетителей, у которых есть ЛК

        ·   Вывести топ-5 страниц сайта по показателю общего кол-ва кликов на данной странице

        ·   Добавить столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)

        ·   Вывести временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.

        ·   Создать второй фрейм данных, который будет содержать информацию о ЛК посетителя сайта со следующим списком атрибутов: 
                1.       Id – уникальный идентификатор личного кабинета
                2.       User_id – уникальный идентификатор посетителя
                3.       ФИО посетителя
                4.       Дату рождения посетителя 
                5.       Дата создания ЛК

* По второму фрейму данных:

        ·   Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.

        ·   Выведите 10% ЛК, у которых максимальная разница между датой создания ЛК и датой последнего посещения.

        ·   Вывести топ-5 страниц, которые чаще всего посещают мужчины и топ-5 страниц, которые посещают чаще женщины.

In [1]:
!pip install pyspark

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
conf = SparkConf()
conf.setMaster("local").setAppName('My app')
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)
print('Запущен Spark версии', spark.version)

Запущен Spark версии 3.3.1


In [3]:
#import functions with alias
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.functions import udf 
from pyspark.sql.window import Window
import datetime

In [4]:
# Запустим Spark сесиию
spark = SparkSession.builder.master("local").\
                    appName("Home_Work_3.1").\
                    config("spark.driver.bindAddress","localhost").\
                    config("spark.ui.port","4040").\
                    getOrCreate()

In [5]:
# Создадим струкутуру данных
schema_web = T.StructType([
                T.StructField("id", T.IntegerType(), False),
                T.StructField("timestamp", T.LongType(), True),
                T.StructField("type", T.StringType(), True),
                T.StructField("page_id", T.IntegerType(), True),
                T.StructField("tag", T.StringType(), True),
                T.StructField("sign", T.BooleanType(), True)])

data_web = [  (1, 1668967844, 'click', 30, 'sport', True), 
              (2, 1668477844,'scroll', 40, 'medical', False),
              (3, 1668967841,'click', 40, 'politic', True),
              (4, 1668989844,'', 40, 'sport', False),
              (5, 1668912843, 'scroll', 10, 'medical', False),
              (6, 1668237845,'visit', 10, 'politic', True),
              (7, 1668967844,'move', 20, 'medical', True),
              (8, 1668967856, 'visit', 30, 'sport', True),
              (9, 1668967844,'click', 30, 'politic', True),
              (10, 1668968878, 'visit', 30, 'sport', True),
              (1, 1668967844, 'visit', 30, 'sport', True),
              (10, 1668935678,'scroll', 40, 'medical', True),
              (10, 1668967844,'click', 40, 'politic', True),
              (4, 1668346777,'visit', 20, 'medical', False),
              (5, 1668456868, 'scroll', 10, 'medical', False),
              (5, 1668456484,'move', 10, 'politic', False),
              (7, 1668967844,'move', 20, 'medical', True),
              (10, 1666778844, 'visit', 30, 'sport', True),
              (9, 1668645644,'move', 30, 'politic', True),
              (1, 1668577844, 'visit', 30, 'sport', True),
              (6, 1668547444,'scroll', 40, 'politic', True),
              (7, 1666568474,'scroll', 10, 'medical', True),
              (10, 1661478494, 'visit', 10, 'sport', True),
              (9, 1668196744,'visit', 10, 'politic', True),
              (1, 1668197890, 'visit', 10, 'politic', True)
            ]

In [6]:
df_web = spark.createDataFrame(data=data_web, schema = schema_web) 
df_web.show()

+---+----------+------+-------+-------+-----+
| id| timestamp|  type|page_id|    tag| sign|
+---+----------+------+-------+-------+-----+
|  1|1668967844| click|     30|  sport| true|
|  2|1668477844|scroll|     40|medical|false|
|  3|1668967841| click|     40|politic| true|
|  4|1668989844|      |     40|  sport|false|
|  5|1668912843|scroll|     10|medical|false|
|  6|1668237845| visit|     10|politic| true|
|  7|1668967844|  move|     20|medical| true|
|  8|1668967856| visit|     30|  sport| true|
|  9|1668967844| click|     30|politic| true|
| 10|1668968878| visit|     30|  sport| true|
|  1|1668967844| visit|     30|  sport| true|
| 10|1668935678|scroll|     40|medical| true|
| 10|1668967844| click|     40|politic| true|
|  4|1668346777| visit|     20|medical|false|
|  5|1668456868|scroll|     10|medical|false|
|  5|1668456484|  move|     10|politic|false|
|  7|1668967844|  move|     20|medical| true|
| 10|1666778844| visit|     30|  sport| true|
|  9|1668645644|  move|     30|pol

In [7]:
# дерево нашей схемы
df_web.printSchema()

root
 |-- id: integer (nullable = false)
 |-- timestamp: long (nullable = true)
 |-- type: string (nullable = true)
 |-- page_id: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- sign: boolean (nullable = true)



In [8]:
# Переведем колонку "timestamp" в тип _unixtime, переименуем колонку на 'event_time' и выведем новую таблицу со всеми колонками
df_web = df_web.select(*[i for i in df_web.columns if i != 'timestamp'],
    F.from_unixtime('timestamp').alias('event_time'))
df_web.show(10)

+---+------+-------+-------+-----+-------------------+
| id|  type|page_id|    tag| sign|         event_time|
+---+------+-------+-------+-----+-------------------+
|  1| click|     30|  sport| true|2022-11-20 21:10:44|
|  2|scroll|     40|medical|false|2022-11-15 05:04:04|
|  3| click|     40|politic| true|2022-11-20 21:10:41|
|  4|      |     40|  sport|false|2022-11-21 03:17:24|
|  5|scroll|     10|medical|false|2022-11-20 05:54:03|
|  6| visit|     10|politic| true|2022-11-12 10:24:05|
|  7|  move|     20|medical| true|2022-11-20 21:10:44|
|  8| visit|     30|  sport| true|2022-11-20 21:10:56|
|  9| click|     30|politic| true|2022-11-20 21:10:44|
| 10| visit|     30|  sport| true|2022-11-20 21:27:58|
+---+------+-------+-------+-----+-------------------+
only showing top 10 rows



d.1 Выведем топ-5 самых активных посетителей сайта

In [9]:
df_web.groupBy('id').agg(F.count("*").alias('event_count')).orderBy('event_count', ascending=False).show(5) 

+---+-----------+
| id|event_count|
+---+-----------+
| 10|          5|
|  1|          4|
|  9|          3|
|  5|          3|
|  7|          3|
+---+-----------+
only showing top 5 rows



d.2 Посчитаем процент посетителей, у которых есть ЛК

In [10]:
# Выедем ID пользователя и инфу о ЛК
all = df_web.select('id','sign').distinct().orderBy("id")
all.show() 

+---+-----+
| id| sign|
+---+-----+
|  1| true|
|  2|false|
|  3| true|
|  4|false|
|  5|false|
|  6| true|
|  7| true|
|  8| true|
|  9| true|
| 10| true|
+---+-----+



In [11]:
# # Выедем ID пользователя, у которых есть ЛК
lk_true = df_web.select('id','sign').distinct().orderBy("id").filter((F.col('sign')=='true'))
lk_true.show()

+---+----+
| id|sign|
+---+----+
|  1|true|
|  3|true|
|  6|true|
|  7|true|
|  8|true|
|  9|true|
| 10|true|
+---+----+



In [12]:
print(f'Процент посетителей, у которых есть ЛК: {round(lk_true.count()/all.count()*100, 2)}%')

Процент посетителей, у которых есть ЛК: 70.0%


d.3 Выведем топ-5 страниц сайта по показателю общего кол-ва кликов на данной странице

In [13]:
# Выведем строки таблицы, где тип события - это клик
df_web.filter((F.col('type')=='click')).show() 

+---+-----+-------+-------+----+-------------------+
| id| type|page_id|    tag|sign|         event_time|
+---+-----+-------+-------+----+-------------------+
|  1|click|     30|  sport|true|2022-11-20 21:10:44|
|  3|click|     40|politic|true|2022-11-20 21:10:41|
|  9|click|     30|politic|true|2022-11-20 21:10:44|
| 10|click|     40|politic|true|2022-11-20 21:10:44|
+---+-----+-------+-------+----+-------------------+



In [14]:
#топ-5 страниц сайта по показателю общего кол-ва кликов на данной странице
df_web.filter((F.col('type')=='click')).groupBy('page_id').count().orderBy("count", acsending = False).show(5)


+-------+-----+
|page_id|count|
+-------+-----+
|     40|    2|
|     30|    2|
+-------+-----+



d.4 Добавим столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)

In [15]:
# У нас будет 1 диапазон: 0-4ч, 2 диапазон: 4-8 и тд...
df_web = df_web.withColumn('time_range', F.ceil((F.hour('event_time')*60+F.minute('event_time'))/240))
df_web.show(15)

+---+------+-------+-------+-----+-------------------+----------+
| id|  type|page_id|    tag| sign|         event_time|time_range|
+---+------+-------+-------+-----+-------------------+----------+
|  1| click|     30|  sport| true|2022-11-20 21:10:44|         6|
|  2|scroll|     40|medical|false|2022-11-15 05:04:04|         2|
|  3| click|     40|politic| true|2022-11-20 21:10:41|         6|
|  4|      |     40|  sport|false|2022-11-21 03:17:24|         1|
|  5|scroll|     10|medical|false|2022-11-20 05:54:03|         2|
|  6| visit|     10|politic| true|2022-11-12 10:24:05|         3|
|  7|  move|     20|medical| true|2022-11-20 21:10:44|         6|
|  8| visit|     30|  sport| true|2022-11-20 21:10:56|         6|
|  9| click|     30|politic| true|2022-11-20 21:10:44|         6|
| 10| visit|     30|  sport| true|2022-11-20 21:27:58|         6|
|  1| visit|     30|  sport| true|2022-11-20 21:10:44|         6|
| 10|scroll|     40|medical| true|2022-11-20 12:14:38|         4|
| 10| clic

d.5 Выведем временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.

In [16]:
df_web.groupBy('time_range').count().orderBy('count', ascending = False).show(1)

+----------+-----+
|time_range|count|
+----------+-----+
|         6|   13|
+----------+-----+
only showing top 1 row



Второй фрейм данных

In [17]:
schema_lk = T.StructType([
                T.StructField("id", T.IntegerType(), False),
                T.StructField("user_id", T.IntegerType(), False),
                T.StructField("fio", T.StringType(), True),
                T.StructField("date_birth", T.DateType(), True),
                T.StructField("time_lk", T.DateType(), True)])

data_lk = [(101, 1, 'James Smith', datetime.datetime(1991, 4, 1), datetime.datetime(2021, 4, 1)),
  (102, 2, 'Michael Rose', datetime.datetime(1994, 7, 1), datetime.datetime(2021, 7, 1)),
  (103, 3, 'Robert Williams', datetime.datetime(1998, 6, 11).date(),datetime.datetime(2022, 4, 4)),
  (104, 4, 'Maria Anne Jones', datetime.datetime(1999, 12, 14),datetime.datetime(2020, 2, 11)),
  (105, 5, 'Jen Mary Brown', datetime.datetime(1992, 11, 17), datetime.datetime(2020, 5, 10)),
  (106, 6, 'Jake Ray', datetime.datetime(1991, 4, 1), datetime.datetime(2021, 4, 1)),
  (107, 7, 'Milly Ann Rose', datetime.datetime(1994, 7, 1), datetime.datetime(2021, 7, 1)),
  (108, 8, 'Robert Grey', datetime.datetime(1998, 6, 11).date(),datetime.datetime(2022, 4, 4)),
  (109, 9, 'Rose Anne Jones', datetime.datetime(1999, 12, 14),datetime.datetime(2020, 2, 11)),
  (110, 10, 'Mark White', datetime.datetime(1992, 11, 17), datetime.datetime(2020, 5, 10))
]

In [18]:
df_lk = spark.createDataFrame(data=data_lk, schema = schema_lk)
df_lk.show()

+---+-------+----------------+----------+----------+
| id|user_id|             fio|date_birth|   time_lk|
+---+-------+----------------+----------+----------+
|101|      1|     James Smith|1991-04-01|2021-04-01|
|102|      2|    Michael Rose|1994-07-01|2021-07-01|
|103|      3| Robert Williams|1998-06-11|2022-04-04|
|104|      4|Maria Anne Jones|1999-12-14|2020-02-11|
|105|      5|  Jen Mary Brown|1992-11-17|2020-05-10|
|106|      6|        Jake Ray|1991-04-01|2021-04-01|
|107|      7|  Milly Ann Rose|1994-07-01|2021-07-01|
|108|      8|     Robert Grey|1998-06-11|2022-04-04|
|109|      9| Rose Anne Jones|1999-12-14|2020-02-11|
|110|     10|      Mark White|1992-11-17|2020-05-10|
+---+-------+----------------+----------+----------+



In [19]:
# дерево нашей схемы
df_lk.printSchema()

root
 |-- id: integer (nullable = false)
 |-- user_id: integer (nullable = false)
 |-- fio: string (nullable = true)
 |-- date_birth: date (nullable = true)
 |-- time_lk: date (nullable = true)



Соединим две таблицы. Тип соединения 'outer', так как нам нужны все данные о посещениях.

In [20]:
df_all = df_lk.alias('lk').join(df_web.alias('web'),
                               on = F.col('lk.user_id') == F.col('web.id'),
                               how = 'outer')
df_all.show(30)

+---+-------+----------------+----------+----------+---+------+-------+-------+-----+-------------------+----------+
| id|user_id|             fio|date_birth|   time_lk| id|  type|page_id|    tag| sign|         event_time|time_range|
+---+-------+----------------+----------+----------+---+------+-------+-------+-----+-------------------+----------+
|101|      1|     James Smith|1991-04-01|2021-04-01|  1| click|     30|  sport| true|2022-11-20 21:10:44|         6|
|101|      1|     James Smith|1991-04-01|2021-04-01|  1| visit|     30|  sport| true|2022-11-20 21:10:44|         6|
|101|      1|     James Smith|1991-04-01|2021-04-01|  1| visit|     30|  sport| true|2022-11-16 08:50:44|         3|
|101|      1|     James Smith|1991-04-01|2021-04-01|  1| visit|     10|politic| true|2022-11-11 23:18:10|         6|
|102|      2|    Michael Rose|1994-07-01|2021-07-01|  2|scroll|     40|medical|false|2022-11-15 05:04:04|         2|
|103|      3| Robert Williams|1998-06-11|2022-04-04|  3| click| 

d.6 Выведем фамилии посетителей, которые читали хотя бы одну новость про спорт.

In [21]:
# Пользовательская функция, которая вычленяет фамилию из полного имени клиента
@udf(T.StringType())
def calc_surname(fio):
    name, *mid_name, surname = fio.split(' ')
    return (surname)

In [22]:
# Выведем фамилии посетителей, которые читали хотя бы одну новость про спорт, ну и сам tag для наглядности
df_all.withColumn('surname', calc_surname(F.col('fio')))\
    .where(F.col('tag') == 'sport')\
    .select('surname','tag')\
    .distinct()\
    .show()

+-------+-----+
|surname|  tag|
+-------+-----+
|   Grey|sport|
|  White|sport|
|  Jones|sport|
|  Smith|sport|
+-------+-----+



d.7 Выведем 10% ЛК, у которых максимальная разница между датой создания ЛК и датой последнего посещения.

In [23]:
n = round(df_lk.count() * 0.1)
n

1

In [24]:
df_all.withColumn("date_diff", F.floor(F.datediff(F.col('event_time'), F.col('time_lk'))))\
    .groupBy('user_id','fio')\
    .agg(F.max('date_diff').alias('max_diff'))\
    .orderBy('max_diff', ascending = False)\
    .select('user_id', 'fio', 'max_diff')\
    .show(n)

+-------+----------------+--------+
|user_id|             fio|max_diff|
+-------+----------------+--------+
|      4|Maria Anne Jones|    1014|
+-------+----------------+--------+
only showing top 1 row



Однако, 10 % из 10 пользователей это всего один человек. Давайте выведем 50%. Или я задание неверно истолковала...

In [25]:
df_all.withColumn("date_diff", F.floor(F.datediff(F.col('event_time'), F.col('time_lk'))))\
    .groupBy('user_id','fio')\
    .agg(F.max('date_diff').alias('max_diff'))\
    .orderBy('max_diff', ascending = False)\
    .select('user_id', 'fio', 'max_diff')\
    .show(round(df_lk.count() * 0.5))

+-------+----------------+--------+
|user_id|             fio|max_diff|
+-------+----------------+--------+
|      4|Maria Anne Jones|    1014|
|      9| Rose Anne Jones|    1013|
|     10|      Mark White|     924|
|      5|  Jen Mary Brown|     924|
|      1|     James Smith|     598|
+-------+----------------+--------+
only showing top 5 rows



d.8 Выведем топ-5 страниц, которые чаще всего посещают мужчины и топ-5 страниц, которые посещают чаще женщины.

In [26]:
# Пользовательская функция, которая определяет пол клиента
@udf(T.StringType())
def calc_gender(fio):
    name, *mid_name, surname = fio.split(' ')
    if name in ['Maria', 'Rose', 'Jen', 'Milly']:
        return 'female'
    else:
        return 'male'

In [27]:
# Проверим, как работает функция: выведем фио и пол
df_all = df_all.withColumn('gender', calc_gender(F.col('fio')))
df_all.select('user_id', 'fio', 'gender').distinct().orderBy('user_id').show()


+-------+----------------+------+
|user_id|             fio|gender|
+-------+----------------+------+
|      1|     James Smith|  male|
|      2|    Michael Rose|  male|
|      3| Robert Williams|  male|
|      4|Maria Anne Jones|female|
|      5|  Jen Mary Brown|female|
|      6|        Jake Ray|  male|
|      7|  Milly Ann Rose|female|
|      8|     Robert Grey|  male|
|      9| Rose Anne Jones|female|
|     10|      Mark White|  male|
+-------+----------------+------+



In [28]:
# Выведем топ-5 страниц, которые чаще всего посещают мужчины
df_all.show(8)

+---+-------+----------------+----------+----------+---+------+-------+-------+-----+-------------------+----------+------+
| id|user_id|             fio|date_birth|   time_lk| id|  type|page_id|    tag| sign|         event_time|time_range|gender|
+---+-------+----------------+----------+----------+---+------+-------+-------+-----+-------------------+----------+------+
|101|      1|     James Smith|1991-04-01|2021-04-01|  1| click|     30|  sport| true|2022-11-20 21:10:44|         6|  male|
|101|      1|     James Smith|1991-04-01|2021-04-01|  1| visit|     30|  sport| true|2022-11-20 21:10:44|         6|  male|
|101|      1|     James Smith|1991-04-01|2021-04-01|  1| visit|     30|  sport| true|2022-11-16 08:50:44|         3|  male|
|101|      1|     James Smith|1991-04-01|2021-04-01|  1| visit|     10|politic| true|2022-11-11 23:18:10|         6|  male|
|102|      2|    Michael Rose|1994-07-01|2021-07-01|  2|scroll|     40|medical|false|2022-11-15 05:04:04|         2|  male|
|103|   

In [29]:
# Выведем топ-5 страниц, которые чаще всего посещают мужчины
df_all.groupby('page_id', 'gender')\
    .agg(F.count("*").alias('page_count'))\
    .orderBy('page_count', ascending = False)\
    .filter(F.col('gender') == 'male')\
    .show(5)

+-------+------+----------+
|page_id|gender|page_count|
+-------+------+----------+
|     30|  male|         6|
|     40|  male|         5|
|     10|  male|         3|
+-------+------+----------+



In [30]:
# Выведем топ-5 страниц, которые чаще всего посещают женщины
df_all.groupby('page_id', 'gender')\
    .agg(F.count("*").alias('page_count'))\
    .orderBy('page_count', ascending = False)\
    .filter(F.col('gender') == 'female')\
    .show(5)

+-------+------+----------+
|page_id|gender|page_count|
+-------+------+----------+
|     10|female|         5|
|     20|female|         3|
|     30|female|         2|
|     40|female|         1|
+-------+------+----------+



Вариант решения этой же задачи, но с оконной функцией.

In [31]:
df_all.groupby('page_id', 'gender')\
    .agg(F.count("*").alias('page_count'))\
    .orderBy('page_count', ascending = False)\
    .withColumn('max_cnt', F.max(F.col('page_count'))\
               .over(Window.partitionBy('gender')))\
    .filter(F.col('page_count') == F.col('max_cnt'))\
    .show(10) 

+-------+------+----------+-------+
|page_id|gender|page_count|max_cnt|
+-------+------+----------+-------+
|     10|female|         5|      5|
|     30|  male|         6|      6|
+-------+------+----------+-------+



## Часть 2 


**Задача:**

* Создать в Postgres таблицы аналогичной структуры

* Создать витрину данных по собственному желанию

In [39]:
!pip install psycopg2

In [40]:
import psycopg2

conn = psycopg2.connect(
    database="airflow",
    user="airflow",
    password="airflow",
    host="127.0.0.1",
    port="35432"
)

cur = conn.cursor()


Создадим первый фрейм данных

In [41]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS PAGE (
  id INTEGER NOT NULL, 
  event_time BIGINT,
  type TEXT,
  page_id INTEGER NOT NULL, 
  tag TEXT, 
  sign BOOL
)
"""

conn.autocommit = True
cur.execute(create_posts_table)

Наполним его значениями

In [42]:
data_web_records = ", ".join(["%s"] * len(data_web))

insert_query = (
    f"INSERT INTO PAGE (id, event_time, type, page_id, tag, sign) VALUES {data_web_records}"
)

conn.autocommit = True
cur.execute(insert_query, data_web)

In [43]:
data_web_records

'%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s'

Создадим второй фрейм данных

In [44]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS DIR__ (
  id SERIAL PRIMARY KEY, 
  user_id INTEGER NOT NULL, 
  fio TEXT NOT NULL, 
  date_birth TIMESTAMP,
  time_lk TIMESTAMP
)
"""
conn.autocommit = True
cur.execute(create_posts_table)

Наполним его значениями

In [45]:
data_lk_records = ", ".join(["%s"] * len(data_lk))

insert_query = (
    f"INSERT INTO DIR__ (id, user_id, fio, date_birth, time_lk) VALUES {data_lk_records}"
)

conn.autocommit = True
cur = conn.cursor()
cur.execute(insert_query, data_lk)

In [46]:
#  Посмотрим на данные в таблице DIRR
cur.execute("SELECT * FROM DIR__ ")
dirs = cur.fetchall()
for row in dirs:
    print(row)

(101, 1, 'James Smith', datetime.datetime(1991, 4, 1, 0, 0), datetime.datetime(2021, 4, 1, 0, 0))
(102, 2, 'Michael Rose', datetime.datetime(1994, 7, 1, 0, 0), datetime.datetime(2021, 7, 1, 0, 0))
(103, 3, 'Robert Williams', datetime.datetime(1998, 6, 11, 0, 0), datetime.datetime(2022, 4, 4, 0, 0))
(104, 4, 'Maria Anne Jones', datetime.datetime(1999, 12, 14, 0, 0), datetime.datetime(2020, 2, 11, 0, 0))
(105, 5, 'Jen Mary Brown', datetime.datetime(1992, 11, 17, 0, 0), datetime.datetime(2020, 5, 10, 0, 0))
(106, 6, 'Jake Ray', datetime.datetime(1991, 4, 1, 0, 0), datetime.datetime(2021, 4, 1, 0, 0))
(107, 7, 'Milly Ann Rose', datetime.datetime(1994, 7, 1, 0, 0), datetime.datetime(2021, 7, 1, 0, 0))
(108, 8, 'Robert Grey', datetime.datetime(1998, 6, 11, 0, 0), datetime.datetime(2022, 4, 4, 0, 0))
(109, 9, 'Rose Anne Jones', datetime.datetime(1999, 12, 14, 0, 0), datetime.datetime(2020, 2, 11, 0, 0))
(110, 10, 'Mark White', datetime.datetime(1992, 11, 17, 0, 0), datetime.datetime(2020, 5, 

In [47]:
# Выберем id пользователя, пол, номер личного кабинета
cur.execute("SELECT DISTINCT(d.user_id) AS id, \
                   CASE WHEN d.fio LIKE ANY (ARRAY['%Maria%', '%Rose%', '%Jen%', '%Milly%']) THEN 'female'\
                         ELSE 'male' END as gender,\
                   d.id AS lk_id\
            FROM page AS p LEFT JOIN dir__ AS d ON p.id=d.user_id\
            ORDER BY id\
            LIMIT(20)")
            
            
dirs = cur.fetchall()            
for row in dirs:
    print(row) 

(1, 'male', 101)
(2, 'female', 102)
(3, 'male', 103)
(4, 'female', 104)
(5, 'female', 105)
(6, 'male', 106)
(7, 'female', 107)
(8, 'male', 108)
(9, 'female', 109)
(10, 'male', 110)


In [48]:
# Выведем id пользователя, новостную тематику, количесвто ее просмотров
#и максимальное количество просмотров одной новости у данного пользователя с помощью оконной функции
cur.execute("SELECT id, tag, count, max(count) OVER (PARTITION BY id) from \
            (SELECT id, TAG, COUNT(tag) AS count FROM page GROUP BY id,TAG ORDER BY ID ASC, count DESC) as t_1")
dirs = cur.fetchall()
for row in dirs:
    print(row)

(1, 'sport', 18, 18)
(1, 'politic', 6, 18)
(2, 'medical', 6, 6)
(3, 'politic', 6, 6)
(4, 'medical', 6, 6)
(4, 'sport', 6, 6)
(5, 'medical', 12, 12)
(5, 'politic', 6, 12)
(6, 'politic', 12, 12)
(7, 'medical', 18, 18)
(8, 'sport', 6, 6)
(9, 'politic', 18, 18)
(10, 'sport', 18, 18)
(10, 'politic', 6, 18)
(10, 'medical', 6, 18)


In [49]:
# Выведем id пользователя, и количество посещений этим пользователем каких-либо интернет страниц
cur.execute("SELECT id, sum(count)::int AS page_count FROM \
                (SELECT id, page_id, count(page_id) as count FROM page GROUP BY id, page_id ORDER BY ID ASC, count DESC)\
                            AS t_1 GROUP BY id")
dirs = cur.fetchall()
for row in dirs:
    print(row)

(1, 24)
(2, 6)
(3, 6)
(4, 12)
(5, 18)
(6, 12)
(7, 18)
(8, 6)
(9, 18)
(10, 30)


In [50]:
cur.close()
conn.close()